# Load Packages and other prerequisite

In [3]:
import sys 
import os
import re
from datetime import datetime
from itertools import (combinations, combinations_with_replacement,
                       permutations, product)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.sparse as sp
import seaborn as sns
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
from regpyhdfe import Regpyhdfe
from scipy.stats import (beta, binom, dirichlet, expon, lognorm,
                         multivariate_normal, norm, pareto)
from tqdm import notebook, trange
import cvxpy as cp
import pickle 

pd.set_option('display.max_columns', None)

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)
if parent_directory not in sys.path:
    sys.path.append(parent_directory)
from config import directory_path
os.chdir(directory_path)
os.getcwd()



'C:\\Users\\nadav\\Dropbox\\uchicago_fourth\\uncertaintyInequality'

## Loading Custom Functions

In [ ]:
%load_ext autoreload
%autoreload 2

In [7]:
import bin2.setGlobals as gl 
from bin2.prePrcoessingFunc import * 

# global torch_device
# torch_device = gl.torch_device


In [ ]:

def add_lagged_variables(df, num_lags, column_list, id_column, date_column):
    """
    Adds lagged variables for specified columns in a pandas DataFrame.
    
    Parameters:
    - df (pd.DataFrame): The original DataFrame.
    - num_lags (int): Number of lagged periods to create.
    - column_list (list of str): List of column names to create lagged variables for.
    - id_column (str): The name of the column used to identify groups for lagging.
    - date_column (str): The name of the date column used for sorting before creating lags.
    
    Returns:
    - pd.DataFrame: A DataFrame including the original data along with the new lagged variables.
    """
    # Make a copy of the original dataframe to avoid modifying it directly
    df_lagged = df.copy()
    # Create a list to store the lag variables
    lag_vars = []
    # Loop through each column in column_list to create lagged versions
    for col in notebook.tqdm(column_list, desc="Col number"):
        # For each column, create lagged versions for the specified number of lags
        for lag in notebook.tqdm(range(1, num_lags + 1), desc="lags number"):
            # Group by id_column, shift the column by 'lag', rename it, and add to the list
            lag_vars.append(df_lagged.groupby(id_column)[col].shift(lag).rename(f"{col}_lag{lag}"))
    # Join the lag variables to the original DataFrame
    df_lagged = df_lagged.join(pd.concat(lag_vars, axis=1))
    return df_lagged

def createLaggedData(df, num_lags, column_list, id_column, date_column, num_chucks=4):
    """
    Splits the DataFrame into chunks, applies lagged variable creation on each chunk, and combines them back.
    
    Parameters:
    - df (pd.DataFrame): The original DataFrame.
    - num_lags (int): Number of lagged periods to create.
    - column_list (list of str): List of column names to create lagged variables for.
    - id_column (str): The name of the column used to identify groups for lagging.
    - date_column (str): The name of the date column used for sorting before creating lags.
    - num_chucks (int): Number of chunks to split the DataFrame into for processing.
    
    Returns:
    - pd.DataFrame: A DataFrame including the original data along with new lagged variables, processed in chunks.
    """
    # Copy the DataFrame to avoid modifying the original
    df_lagged = df.copy()
    # Split the DataFrame into manageable chunks based on unique values in 'HH_ID'
    ids = df_lagged[id_column].unique().tolist()
    chunk_size = len(ids) // num_chucks
    splitted_ids = [ids[i:i + chunk_size] for i in range(0, len(ids), chunk_size)]
    # Initialize a list to store DataFrames with lagged variables from each chunk
    laggedDataframes = []
    # Process each chunk separately
    for l in notebook.tqdm(splitted_ids, desc="Chunks"):
        # Filter the DataFrame for the current chunk's IDs
        sm_df = df_lagged[df_lagged[id_column].isin(l)]
        # Ensure data is complete and sorted before adding lags
        sm_df = sm_df.set_index([*id_column, date_column]).unstack(fill_value=".").stack().sort_index(level=1).reset_index()
        sm_df = sm_df.sort_values(["HH_ID", "MONTH"])
        # Add lagged variables to the chunk
        sm_df = add_lagged_variables(sm_df, num_lags, column_list, id_column, date_column)
        # Replace placeholder with NaN and drop rows with missing values
        sm_df = sm_df.replace(".", np.nan)
        sm_df.dropna(inplace=True)
        # Add the processed chunk to the list
        laggedDataframes.append(sm_df)
    # Combine all processed chunks into a single DataFrame
    df_lagged = pd.concat(laggedDataframes, axis=0)
    return df_lagged


# Load Data

- Income Panel
- Consumption Panel

In [5]:
incomePanel = pd.read_feather(gl.procDataFolder + '\HH_Income_Panel_long.feather')
incomePanel['MONTH'] = pd.to_datetime(incomePanel['MONTH'], format='%b %Y')


In [6]:
incomePanelFilter = incomePanel[['HH_ID','MONTH','MONTH_SLOT','FAMILY_SHIFTED','REASON_FOR_NON_RESPONSE']].copy()
incomePanelFilter.sort_values(['HH_ID','MONTH'],inplace=True)
incomePanelFilter.loc[incomePanelFilter['FAMILY_SHIFTED'].isnull(),'FAMILY_SHIFTED'] = "N"


In [7]:
incomePanelFilter = incomePanelFilter[incomePanelFilter['REASON_FOR_NON_RESPONSE']=="No Failure"].copy()
incomePanelFilter['famShiftInd']=0
incomePanelFilter.loc[incomePanelFilter["FAMILY_SHIFTED"]=='Y',"famShiftInd"]=1
incomePanelFilter['numFamilyShifts'] = incomePanelFilter.groupby('HH_ID')['famShiftInd'].cumsum()
incomePanelFilter = incomePanelFilter[['HH_ID','MONTH','numFamilyShifts']].copy()


In [8]:
consumptionData = pd.read_feather(gl.procDataFolder + 'HH_consumption_long.feather')
consumptionData['MONTH'] = pd.to_datetime(consumptionData['MONTH'], format='%b %Y')

In [9]:
consumptionPanelFilter = consumptionData[['HH_ID','MONTH','MONTH_SLOT','FAMILY_SHIFTED','REASON_FOR_NON_RESPONSE']].copy()
consumptionPanelFilter.sort_values(['HH_ID','MONTH'],inplace=True)
consumptionPanelFilter.loc[consumptionPanelFilter['FAMILY_SHIFTED'].isnull(),'FAMILY_SHIFTED'] = "N"


In [10]:
consumptionPanelFilter = consumptionPanelFilter[consumptionPanelFilter['REASON_FOR_NON_RESPONSE']=="No Failure"].copy()
consumptionPanelFilter['famShiftInd']=0
consumptionPanelFilter.loc[consumptionPanelFilter["FAMILY_SHIFTED"]=='Y',"famShiftInd"]=1
consumptionPanelFilter['numFamilyShifts'] = consumptionPanelFilter.groupby('HH_ID')['famShiftInd'].cumsum()
consumptionPanelFilter = consumptionPanelFilter[['HH_ID','MONTH','numFamilyShifts']].copy()


In [11]:
filter = incomePanelFilter.merge(consumptionPanelFilter,on=['HH_ID','MONTH'],how='inner',validate='1:1')
del [incomePanelFilter ,consumptionPanelFilter]
filter= filter[['HH_ID','MONTH','numFamilyShifts_x']].copy()


In [12]:
## This Does not do anything. i.e. we get the same results in both DB
# print(len(filter['HH_ID'].unique()))
# mask = filter.groupby('HH_ID').apply(lambda x: x['numFamilyShifts_x'].ne(x['numFamilyShifts_y']).any())
# filter = filter[filter['HH_ID'].isin(mask[~mask].index)]
# print(len(filter['HH_ID'].unique()))



In [13]:
incomeConsumptionPanel = filter.merge(consumptionData,on=['HH_ID','MONTH'],how="inner",validate="1:1")
incomeConsumptionPanel = incomeConsumptionPanel.merge(incomePanel,on=['HH_ID','MONTH'],how="inner",validate="1:1")
del [consumptionData,incomePanel,filter]

## Import Wealth Data (estimated in Stata using RegHDFE)

**Note:** the total capital here is constructed by adding to the inital capital total labor income and TOTAL income from interest. In what follows I'm actually not going to use the realized interest (what's positive returns to negative assets? )

In [14]:
#Load Data 
wealthData = pd.read_csv(gl.procDataFolder + '\wealthDistribution.csv')
# Keep the capital that was generated given the 11 month before the end of 2019 
wealthData = wealthData[['HH_ID','MONTH','capitalt_12']]
wealthData.rename({'capitalt_12':'aPreviousT'},axis=1,inplace=True)
wealthData = wealthData[~wealthData['aPreviousT'].isnull()]
wealthData['MONTH'] = pd.to_datetime(wealthData['MONTH'], format='%b %Y')
wealthData.rename({'aPreviousT':'initalCapital'},inplace=True,axis=1) 

# notice that in the previous iteration I took the first capital (i.e. the one from 201501). This was fine because everyone start at there. Now i'm taking the most recent capital. i.e. At for predicting At+1. so yeah. I would need to change that updating rule. 
# It's also makes sense. 

In [15]:
incomeConsumptionPanel = incomeConsumptionPanel.merge(wealthData,on=['HH_ID','MONTH'],how="left",validate="1:1")

#I need to correct for capital at -99

In [16]:

incomeConsumptionPanel['labourIncome'] = (incomeConsumptionPanel['INCOME_OF_ALL_MEMBERS_FROM_WAGES'] + 
incomeConsumptionPanel['INCOME_OF_ALL_MEMBERS_FROM_PENSION'] + incomeConsumptionPanel['INCOME_OF_HOUSEHOLD_FROM_SELF_PRODUCTION'] + 
incomeConsumptionPanel['INCOME_OF_HOUSEHOLD_FROM_GOVERNMENT_TRANSFERS'] + 
incomeConsumptionPanel['INCOME_OF_HOUSEHOLD_FROM_GAMBLING'])

incomeConsumptionPanel = incomeConsumptionPanel[['HH_ID','MONTH','numFamilyShifts_x','initalCapital','labourIncome','TOTAL_EXPENDITURE']].copy()
incomeConsumptionPanel.sort_values(['HH_ID','MONTH'],inplace=True)

In [ ]:
incomeConsumptionPanel = createLaggedData(df=incomeConsumptionPanel,num_lags=48,column_list=['labourIncome','TOTAL_EXPENDITURE'],
                                        id_column=["HH_ID",'numFamilyShifts_x'],date_column="MONTH",num_chucks=4) #Notice that the first item in ID column must be HH_ID. 


## Add HH chars

### Add Age

In [20]:
HHAgePanel = pd.read_feather(gl.procDataFolder + '\HoH_Ages_longPanel.feather')
HHAgePanel['MONTH'] = pd.to_datetime(HHAgePanel['MONTH'], format='%b %Y')
incomeConsumptionPanel = incomeConsumptionPanel.merge(HHAgePanel,on=['HH_ID',"MONTH"],how='left',validate='1:1')
incomeConsumptionPanel.rename({'curAge':'ageMonth'},inplace=True,axis=1)


### Add Caste 

In [21]:
def replace_none_with_values(columns, data):
    result_column = data[columns[0]].copy()
    for col in columns[1:]:
        result_column[(result_column == "Not Stated") | (result_column == "None")| (result_column.isnull())] = data[col][(result_column == "Not Stated") | (result_column == "None")| (result_column.isnull())]
    return result_column

HHPanel = pd.read_feather(gl.procDataFolder + '\HH_Panel.feather')
colList = [i for i in HHPanel.columns if i.find('CASTE_CATEGORY')>-1]
HHPanel['CASTE_CAT'] = replace_none_with_values(colList, HHPanel)
castesIn2015 = HHPanel[['HH_ID','CASTE_CAT']].copy()
castesIn2015.rename({'CASTE_CAT':'CASTE_CATEGORY'},axis=1,inplace=True)
incomeConsumptionPanel = incomeConsumptionPanel.merge(castesIn2015,on="HH_ID",how="left", validate="m:1")
incomeConsumptionPanel.groupby(['CASTE_CATEGORY'],dropna=False).size().reset_index()
#The missing values are really missing 


,CASTE_CATEGORY,0
0,Data Not Available,246
1,Intermediate Caste,24300
2,Not Applicable,325
3,Not Stated,733
4,OBC,89285
5,SC,57356
6,ST,8588
7,Upper Caste,58751
8,NaN,29442


In [ ]:
# incomeConsumptionPanel[incomeConsumptionPanel['ageMonth'].isnull()]['HH_ID']
# These HH are not in the member income data and not in the people of india data. So we can't construct for them an age
# and just to be clear - these HH are not in the raw files. 

In [ ]:
incomeConsumptionPanel.loc[incomeConsumptionPanel['CASTE_CATEGORY']=="ST",'CASTE_CATEGORY']="SC"
dummyVars = pd.get_dummies(incomeConsumptionPanel['CASTE_CATEGORY'])
casteColsName = list(dummyVars.columns)
incomeConsumptionPanel = pd.concat([dummyVars,incomeConsumptionPanel],axis=1)
incomeConsumptionPanel.columns = ['{}{}'.format(c, '' if c not in casteColsName else '_caste') for c in incomeConsumptionPanel.columns]
incomeConsumptionPanel

### Sort Columns  
**NOTE:** When we sort columns it must be first AGE and then WEALTH

In [25]:
def reorder_columns(string_list):
    def sort_key(string):
        match = re.match(r'(.*?)(\d+)$', string)
        if match:
            string_before_num = match.group(1)
            num = int(match.group(2))
            return ( num, string_before_num, num)
        else:
            return (0, string)

    return sorted(string_list, key=sort_key)

colList = reorder_columns(incomeConsumptionPanel.columns)
predictedCols = [ 'labourIncome','TOTAL_EXPENDITURE']
laggedCols = [i for i in colList if i[-1].isdigit()]
laggedCols.reverse()
other_cols = list(filter(lambda x: x not in laggedCols + predictedCols , colList))
df_analysis = incomeConsumptionPanel[other_cols + laggedCols + predictedCols].copy()

In [26]:
list_of_cols = ['HH_ID'] + [c for c in df_analysis.columns if 
                            (c.find('labourIncome')>-1 or  c.find('TOTAL_EXPENDITURE')>-1 or 
                            c.find('ageMonth') > -1 or  
                            c.find('initalCapital') >-1  or 
                            c.find('_caste') >-1 ) ]
                            


df_analysis=df_analysis[list_of_cols]
df_analysis

,HH_ID,Data Not Available_caste,Intermediate Caste_caste,Not Applicable_caste,Not Stated_caste,OBC_caste,SC_caste,Upper Caste_caste,ageMonth,initalCapital,labourIncome_lag48,TOTAL_EXPENDITURE_lag48,labourIncome_lag47,TOTAL_EXPENDITURE_lag47,labourIncome_lag46,TOTAL_EXPENDITURE_lag46,labourIncome_lag45,TOTAL_EXPENDITURE_lag45,labourIncome_lag44,TOTAL_EXPENDITURE_lag44,labourIncome_lag43,TOTAL_EXPENDITURE_lag43,labourIncome_lag42,TOTAL_EXPENDITURE_lag42,labourIncome_lag41,TOTAL_EXPENDITURE_lag41,labourIncome_lag40,TOTAL_EXPENDITURE_lag40,labourIncome_lag39,TOTAL_EXPENDITURE_lag39,labourIncome_lag38,TOTAL_EXPENDITURE_lag38,labourIncome_lag37,TOTAL_EXPENDITURE_lag37,labourIncome_lag36,TOTAL_EXPENDITURE_lag36,labourIncome_lag35,TOTAL_EXPENDITURE_lag35,labourIncome_lag34,TOTAL_EXPENDITURE_lag34,labourIncome_lag33,TOTAL_EXPENDITURE_lag33,labourIncome_lag32,TOTAL_EXPENDITURE_lag32,labourIncome_lag31,TOTAL_EXPENDITURE_lag31,labourIncome_lag30,TOTAL_EXPENDITURE_lag30,labourIncome_lag29,TOTAL_EXPENDITURE_lag29,labourIncome_lag28,TOTAL_EXPENDITURE_lag28,labourIncome_lag27,TOTAL_EXPENDITURE_lag27,labourIncome_lag26,TOTAL_EXPENDITURE_lag26,labourIncome_lag25,TOTAL_EXPENDITURE_lag25,labourIncome_lag24,TOTAL_EXPENDITURE_lag24,labourIncome_lag23,TOTAL_EXPENDITURE_lag23,labourIncome_lag22,TOTAL_EXPENDITURE_lag22,labourIncome_lag21,TOTAL_EXPENDITURE_lag21,labourIncome_lag20,TOTAL_EXPENDITURE_lag20,labourIncome_lag19,TOTAL_EXPENDITURE_lag19,labourIncome_lag18,TOTAL_EXPENDITURE_lag18,labourIncome_lag17,TOTAL_EXPENDITURE_lag17,labourIncome_lag16,TOTAL_EXPENDITURE_lag16,labourIncome_lag15,TOTAL_EXPENDITURE_lag15,labourIncome_lag14,TOTAL_EXPENDITURE_lag14,labourIncome_lag13,TOTAL_EXPENDITURE_lag13,labourIncome_lag12,TOTAL_EXPENDITURE_lag12,labourIncome_lag11,TOTAL_EXPENDITURE_lag11,labourIncome_lag10,TOTAL_EXPENDITURE_lag10,labourIncome_lag9,TOTAL_EXPENDITURE_lag9,labourIncome_lag8,TOTAL_EXPENDITURE_lag8,labourIncome_lag7,TOTAL_EXPENDITURE_lag7,labourIncome_lag6,TOTAL_EXPENDITURE_lag6,labourIncome_lag5,TOTAL_EXPENDITURE_lag5,labourIncome_lag4,TOTAL_EXPENDITURE_lag4,labourIncome_lag3,TOTAL_EXPENDITURE_lag3,labourIncome_lag2,TOTAL_EXPENDITURE_lag2,labourIncome_lag1,TOTAL_EXPENDITURE_lag1,labourIncome,TOTAL_EXPENDITURE
0,10002477,0,1,0,0,0,0,0,490.0,94759.703,16000.0,9611.0,16000.0,9653.0,16000.0,9456.0,16000.0,9637.0,8500.0,7948.0,8000.0,8100.0,8500.0,7955.0,8000.0,8160.0,12000.0,7473.0,12000.0,8405.0,12000.0,8105.0,12000.0,10560.0,9000.0,8910.0,9000.0,8700.0,9000.0,9010.0,9000.0,10542.0,11600.0,10595.0,12100.0,10995.0,12100.0,11235.0,12600.0,11680.0,9500.0,5024.0,9500.0,6259.0,9500.0,5064.0,9500.0,6169.0,8090.0,14585.0,9090.0,15330.0,8090.0,14585.0,9090.0,18030.0,17093.0,14335.0,16593.0,14030.0,17093.0,14095.0,16593.0,13880.0,9090.0,10565.0,8080.0,11795.0,9090.0,10565.0,8080.0,10735.0,20600.0,11840.0,22650.0,15580.0,24740.0,11880.0,22750.0,11470.0,26120.0,17560.0,29030.0,17190.0,33140.0,18040.0,29030.0,16500.0,35250.0,22930.0,33660.0,17910.0,33070.0,25100.0,34680.0,19300.0,13200.0,13060.0
1,10002477,0,1,0,0,0,0,0,491.0,95889.703,16000.0,9653.0,16000.0,9456.0,16000.0,9637.0,8500.0,7948.0,8000.0,8100.0,8500.0,7955.0,8000.0,8160.0,12000.0,7473.0,12000.0,8405.0,12000.0,8105.0,12000.0,10560.0,9000.0,8910.0,9000.0,8700.0,9000.0,9010.0,9000.0,10542.0,11600.0,10595.0,12100.0,10995.0,12100.0,11235.0,12600.0,11680.0,9500.0,5024.0,9500.0,6259.0,9500.0,5064.0,9500.0,6169.0,8090.0,14585.0,9090.0,15330.0,8090.0,14585.0,9090.0,18030.0,17093.0,14335.0,16593.0,14030.0,17093.0,14095.0,16593.0,13880.0,9090.0,10565.0,8080.0,11795.0,9090.0,10565.0,8080.0,10735.0,20600.0,11840.0,22650.0,15580.0,24740.0,11880.0,22750.0,11470.0,26120.0,17560.0,29030.0,17190.0,33140.0,18040.0,29030.0,16500.0,35250.0,22930.0,33660.0,17910.0,33070.0,25100.0,34680.0,19300.0,13200.0,13060.0,12220.0,11090.0
2,10002477,0,1,0,0,0,0,0,492.0,96449.703,16000.0,9456.0,16000.0,9637.0,8500.0,7948.0,8000.0,8100.0,8500.0,7955.0,8000.0,8160.0,12000.0,7473.0,12000.0,8405.0,12000.0,8105.0,12000.0,10560.0,9000.

### Winsorizing

In [30]:
#Get bounds 
maxValue_Income = np.max(df_analysis[[c for c in df_analysis.columns if c.find('labourIncome')>-1] ].to_numpy()) *1.5
maxValue_Consumption = maxValue_Income
minValue_Income = 0
minValue_Consumtpion = np.percentile(df_analysis[[c for c in df_analysis.columns if c.find('TOTAL_EXPENDITURE')>-1] ].to_numpy(),0.01) 

conCols = [c for c in df_analysis.columns if c.find('TOTAL_EXPENDITURE')>-1] 
for c in conCols:
    df_analysis = Winsorizing(df_analysis,c,maxValue_Consumption,minValue_Consumtpion)


conCols = [c for c in df_analysis.columns if c.find('labourIncome')>-1] 
for c in conCols:
    df_analysis = Winsorizing(df_analysis,c,maxValue_Income,minValue_Income)
    
df_analysis= df_analysis.dropna()
print(df_analysis.shape) #drops ages and capital that are missing 



(242043, 105)


### Clean Caste Data

In [28]:
castesCols = [i for i in df_analysis.columns if i.find('_caste')>-1]
df_analysis[castesCols].describe()


,Data Not Available_caste,Intermediate Caste_caste,Not Applicable_caste,Not Stated_caste,OBC_caste,SC_caste,Upper Caste_caste
count,243812.000000,243812.000000,243812.000000,243812.000000,243812.000000,243812.000000,243812.000000
mean,0.000964,0.098486,0.001333,0.003006,0.362205,0.267809,0.238069
std,0.031031,0.297971,0.036486,0.054748,0.480639,0.442818,0.425902
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
df_analysis = df_analysis[df_analysis['Not Stated_caste']!=1].copy()
df_analysis = df_analysis[df_analysis['Not Applicable_caste']!=1]
df_analysis = df_analysis[df_analysis['Data Not Available_caste']!=1]
print(df_analysis[castesCols].describe())
df_analysis.drop(['Not Stated_caste','Not Applicable_caste','Data Not Available_caste'],axis=1,inplace=True)

       Data Not Available_caste  Intermediate Caste_caste  \
count                  242519.0             242519.000000   
mean                        0.0                  0.099011   
std                         0.0                  0.298677   
min                         0.0                  0.000000   
25%                         0.0                  0.000000   
50%                         0.0                  0.000000   
75%                         0.0                  0.000000   
max                         0.0                  1.000000   

       Not Applicable_caste  Not Stated_caste      OBC_caste       SC_caste  \
count              242519.0          242519.0  242519.000000  242519.000000   
mean                    0.0               0.0       0.364136       0.269237   
std                     0.0               0.0       0.481188       0.443564   
min                     0.0               0.0       0.000000       0.000000   
25%                     0.0               0.0       0.0

# Save Data

In [6]:
Save Data 
with open('proc_data/temps/df_analysis_temp152023.pkl','wb') as f:
    pickle.dump(df_analysis, f)


